## Salary Prediction for Job Postings

This competition is an in-class competition for the course, "Artificial Intelligence and Machine Learning (Fall 2023)", in Renmin University of China.

## Content

- [packages](#pack)
- [load data](#load)
- [engineering](#engineer)
- [model](#model)
- [submit](#sub)

## <a id = 'pack'> Packages </a>

In [49]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import train_test_split
import seaborn as sns
import matplotlib.pyplot as plt


## <a id = 'load'> Load Data </a>

In [50]:
train = pd.read_csv('usjobs_train.csv')
print(train.shape)
train.sample(3)

(33248, 21)


,ID,Job,Jobs_Group,Profile,Remote,Company,Location,City,State,Frecuency_Salary,...,Skills,Sector,Sector_Group,Revenue,Employee,Company_Score,Reviews,Director,Director_Score,URL
32986,job_bcdf1152083e31eb,"Analyst, Business Insights - Shaw Ross (Mirama...",Business Analyst,NaN,NaN,Southern Glazer’s Wine and Spirits,"Miramar, FL 33027",Miramar,FL,year,...,"['PowerPoint', 'Excel', 'SAP', 'Bachelor', 'Of...",Wholesale Trade,Sales,XXXL,XXXL,3.5,1650.0,Wayne Chaplin,0.73,http://southernglazers.com/careers
11337,job_ef6f150f4f104213,Data Engineer I- Institute for Health Equity R...,Data Engineer,NaN,NaN,Mount Sinai,"New York, NY 10029 (Yorkville area)+2 ubicaciones",New York,NY,year,...,"['Python', 'SQL', 'Snowflake', 'Azure', 'Bache...",Hospitals and Health Centers,Health,NaN,NaN,3.8,93.0,NaN,NaN,NaN
24701,job_1cdee9886174a088,"Grants, Statistical Development, and Special P...",Statistician/Mathemathics,NaN,Remote,Commonwealth of PA,Pennsylvania,NaN,PA,year,...,['Office'],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [51]:
test = pd.read_csv('usjobs_test.csv')
print(test.shape)
test.sample(3)

(22166, 20)


,ID,Job,Jobs_Group,Profile,Remote,Company,Location,City,State,Frecuency_Salary,Skills,Sector,Sector_Group,Revenue,Employee,Company_Score,Reviews,Director,Director_Score,URL
3892,job_0f9d85dbb7f6375a,Programmer Analyst I/II,Analyst,NaN,NaN,"Merced County, CA","Merced County, CA+1 location",Merced County,CA,year,['SSIS'],Government and Public Administration,Government,NaN,XXL,3.6,44.0,NaN,NaN,https://www.co.merced.ca.us/
4433,job_33b54321f54d2b60,"Data Analyst, Bureau of Early Intervention",Data Analyst,NaN,NaN,NYC Careers,"Queens, NY 11101+6 ubicaciones",Queens,NY,year,"['Python', 'SQL', 'Tableau', 'Office', 'R', 'V...",Government and Public Administration,Government,NaN,NaN,2.8,10.0,NaN,NaN,NaN
15327,job_cb935ad9d765b0ab,Financial Controller,Controller,NaN,NaN,Providence Management,"Oxford, MS 38655",Oxford,MS,year,"['CMA', 'CPA', 'Office', 'Bachelor']",Hospitals and Health Centers,Health,NaN,NaN,3.9,10.0,NaN,NaN,NaN


In [52]:
submission = pd.read_csv('usjobs_sample_submission (2).csv')
print(submission.shape)
submission.sample(3)

(22166, 2)


,ID,Mean_Salary
5182,job_8418e0bed2607a74,105000
21671,job_f8c890cb2d022407,105000
14699,job_3656a3ab718c5350,105000


## <a id = 'engineer'> Engineering </a>

In [53]:
train.isna().sum()

# In summary, train.isna().sum() will return a Series showing how many missing values (NaNs) are present in each column of your training dataset.

ID                      0
Job                     0
Jobs_Group              0
Profile             21107
Remote              19319
Company                 9
Location               13
City                 3824
State                3112
Frecuency_Salary        0
Mean_Salary             0
Skills                  0
Sector               7214
Sector_Group         7214
Revenue             18318
Employee            12799
Company_Score        8762
Reviews              8762
Director            20785
Director_Score      21924
URL                 16033
dtype: int64

In [54]:
def rearrange_dataset(train_data, test_data):
    #1.train 
    skills_list = []
    for i in range(len(train_data)):
        zip_list = train_data['Skills'][i].replace('[','').replace(']','').replace("'",'').replace(" ",'').split(',')
        dict_temp = {}
        str_ = ''
        for j in range(len(zip_list)):
            dict_temp[zip_list[j]] = 1
            str_ += zip_list[j] + " "
        dict_temp['skills_clean'] = str_    
        skills_list.append(dict_temp)
        
    train_data = pd.concat([train_data,pd.DataFrame(skills_list).fillna(0)],axis =1)  
    
    
    #2.test
    skills_list = []
    for i in range(len(test_data)):
        zip_list = test_data['Skills'][i].replace('[','').replace(']','').replace("'",'').replace(" ",'').split(',')
        dict_temp = {}
        str_ = ''
        for j in range(len(zip_list)):
            dict_temp[zip_list[j]] = 1
            str_ += zip_list[j] + " "
        dict_temp['skills_clean'] = str_        
        skills_list.append(dict_temp)
        
    test_data = pd.concat([test_data,pd.DataFrame(skills_list).fillna(0)],axis =1)      
    
    return train_data, test_data

In [55]:
%%time
train_data1, test_data1 = rearrange_dataset(train_data= train, test_data= test)

CPU times: total: 78.1 ms
Wall time: 607 ms


In [56]:
train_data1[['Job','Company','skills_clean','ArtificialIntelligence', 'MachineLearning', 'Python', 'TensorFlow',
       'Excel', 'Spark', 'PhD', 'AWS', 'C++', 'DeepLearning', 'Java', 'Master',
       'Office', 'Docker', 'Word', 'Azure', 'Hadoop', 'Airflow', 'MBA', 'CPA',
       'Director_Score', 'Snowflake', 'Databricks', 'SQL', '', 'Company_Score',
       'GoogleCloud', 'SciKit', 'Reviews', 'Pandas', 'ChatGPT', 'Agile', 'R',
       'NeuralNetwork', 'Jupyter', 'PowerPoint', 'Spanish', 'NumPy', 'React',
       'GitHub', 'Looker', 'English', 'Access', 'Matplotlib', 'Seaborn', 'C#',
       'Rust', 'VBA', 'Bachelor']].isna().sum()

Job                           0
Company                       9
skills_clean                  0
ArtificialIntelligence        0
MachineLearning               0
Python                        0
TensorFlow                    0
Excel                         0
Spark                         0
PhD                           0
AWS                           0
C++                           0
DeepLearning                  0
Java                          0
Master                        0
Office                        0
Docker                        0
Word                          0
Azure                         0
Hadoop                        0
Airflow                       0
MBA                           0
CPA                           0
Director_Score            21924
Snowflake                     0
Databricks                    0
SQL                           0
                              0
Company_Score              8762
GoogleCloud                   0
SciKit                        0
Reviews 

In [57]:
train_data1['Director_Score'].fillna(np.mean(train_data1['Director_Score']), inplace= True)
train_data1['Company_Score'].fillna(np.mean(train_data1['Company_Score']), inplace= True)
train_data1['Reviews'].fillna(np.mean(train_data1['Reviews']), inplace= True)

C:\Users\Gajen\AppData\Local\Temp\ipykernel_28192\966961151.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  train_data1['Director_Score'].fillna(np.mean(train_data1['Director_Score']), inplace= True)
C:\Users\Gajen\AppData\Local\Temp\ipykernel_28192\966961151.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting 

## <a id = 'model'> Model </a>

In [58]:
X_train, X_test, y_train, y_test = train_test_split(train_data1[['ArtificialIntelligence', 'MachineLearning', 'Python', 'TensorFlow',
       'Excel', 'Spark', 'PhD', 'AWS', 'C++', 'DeepLearning', 'Java', 'Master',
       'Office', 'Docker', 'Word', 'Azure', 'Hadoop', 'Airflow', 'MBA', 'CPA',
       'Director_Score', 'Snowflake', 'Databricks', 'SQL', '', 'Company_Score',
       'GoogleCloud', 'SciKit', 'Reviews']],
                                                   train_data1['Mean_Salary'],
                                                   test_size = 0.2)

In [59]:
clf = RandomForestRegressor(n_estimators= 500,
                            max_depth = 30,
                            min_samples_leaf = 25,
                           n_jobs= 5)

clf.fit(X_train,
       y_train)

RandomForestRegressor(max_depth=30, min_samples_leaf=25, n_estimators=500,
                      n_jobs=5)

In [60]:
mean_absolute_error(y_test,clf.predict(X_test))

26604.747284594734

## <a id = 'sub'> Submission </a>

In [61]:
test_data1['Director_Score'].fillna(np.mean(train_data1['Director_Score']), inplace= True)
test_data1['Company_Score'].fillna(np.mean(train_data1['Company_Score']), inplace= True)
test_data1['Reviews'].fillna(np.mean(train_data1['Reviews']), inplace= True)

C:\Users\Gajen\AppData\Local\Temp\ipykernel_28192\2491048070.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  test_data1['Director_Score'].fillna(np.mean(train_data1['Director_Score']), inplace= True)
C:\Users\Gajen\AppData\Local\Temp\ipykernel_28192\2491048070.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting

In [62]:
test_df = test_data1[['ID','ArtificialIntelligence', 'MachineLearning', 'Python', 'TensorFlow',
       'Excel', 'Spark', 'PhD', 'AWS', 'C++', 'DeepLearning', 'Java', 'Master',
       'Office', 'Docker', 'Word', 'Azure', 'Hadoop', 'Airflow', 'MBA', 'CPA',
       'Director_Score', 'Snowflake', 'Databricks', 'SQL', '', 'Company_Score',
       'GoogleCloud', 'SciKit', 'Reviews']]

In [63]:
pred = clf.predict(test_df.drop('ID', axis = 1))

In [64]:
test_df['Mean_Salary'] = pred

C:\Users\Gajen\AppData\Local\Temp\ipykernel_28192\1862751176.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['Mean_Salary'] = pred


In [65]:
test_df[['ID','Mean_Salary']].to_csv('submission1.csv', index = False)